In [2]:
!pip install findspark pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=520b0c710294788a87ff86ba1fdfcef5a0403f3261de687fc8e743eea45b4f91
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
# Import findspark and initialize.
import findspark
findspark.init()

In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [26]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)




In [31]:
from pyspark.sql.functions import month, year
df.select(year(df["date"]))
df = df.withColumn("year", year(df['date']))
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
|5aa00529-0533-46b...|2019-01-30|      2

In [32]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('houses')


In [33]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
  SELECT
 year,
 round(avg(price) ,2) AS avg_price
  FROM houses
  WHERE bedrooms=4
  GROUP BY year
 """).show()

+----+---------+
|year|avg_price|
+----+---------+
|2022|296363.88|
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
+----+---------+



In [36]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
  SELECT
 date_built,
 round(avg(price) ,2) AS avg_price
  FROM houses
  WHERE bedrooms=3 AND bathrooms =3
  GROUP BY date_built
 """).show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2016|290555.07|
|      2012|293683.19|
|      2017|292676.79|
|      2014|290852.27|
|      2013|295962.27|
|      2011|291117.47|
|      2015| 288770.3|
|      2010|292859.62|
+----------+---------+



In [37]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

spark.sql("""
  SELECT
 date_built,
 round(avg(price) ,2) AS avg_price
  FROM houses
  WHERE bedrooms=3 AND bathrooms =3 AND floors =2 and sqft_living >= 2000
  GROUP BY date_built
 """).show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2016| 293965.1|
|      2012|307539.97|
|      2017|280317.58|
|      2014|298264.72|
|      2013|303676.79|
|      2011|276553.81|
|      2015|297609.97|
|      2010|285010.22|
+----------+---------+



In [38]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

spark.sql("""
  SELECT
 view,
 round(avg(price) ,2) AS avg_price
  FROM houses
  WHERE price  >= 350000
  GROUP BY view
 """).show()


start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|   7| 403005.77|
|  51| 788128.21|
|  15|  404673.3|
|  54| 798684.82|
|  11| 399548.12|
|  29| 397771.65|
|  69| 750537.94|
|  42|  396964.5|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|   3|  398867.6|
|  30|  397862.0|
|  34| 401419.75|
|  59|  791453.0|
|   8| 398592.71|
|  28| 402124.62|
|  22| 402022.68|
|  85|1056336.74|
|  16| 399586.53|
+----+----------+
only showing top 20 rows

--- 0.00016760826110839844 seconds ---


In [39]:
# 7. Cache the the temporary table home_sales.
df.createOrReplaceTempView("home_sales")
spark.catalog.cacheTable("home_sales")

In [40]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [41]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
spark.sql("""
  SELECT
 view,
 round(avg(price) ,2) AS avg_price
  FROM home_sales
  WHERE price  >= 350000
  GROUP BY view
 """).show()


start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|   7| 403005.77|
|  51| 788128.21|
|  15|  404673.3|
|  54| 798684.82|
|  11| 399548.12|
|  29| 397771.65|
|  69| 750537.94|
|  42|  396964.5|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|   3|  398867.6|
|  30|  397862.0|
|  34| 401419.75|
|  59|  791453.0|
|   8| 398592.71|
|  28| 402124.62|
|  22| 402022.68|
|  85|1056336.74|
|  16| 399586.53|
+----+----------+
only showing top 20 rows

--- 9.822845458984375e-05 seconds ---


In [42]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.parquet("parquet")

In [44]:
# 11. Read the formatted parquet data.
parquetFile= spark.read.parquet("parquet")

In [45]:
# 12. Create a temporary table for the parquet data.
parquetFile.createOrReplaceTempView("parquetFile")

In [46]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.


spark.sql("""
  SELECT
 view,
 round(avg(price) ,2) AS avg_price
  FROM parquetFile
  WHERE price  >= 350000
  GROUP BY view
 """).show()

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|   7| 403005.77|
|  51| 788128.21|
|  15|  404673.3|
|  54| 798684.82|
|  11| 399548.12|
|  29| 397771.65|
|  69| 750537.94|
|  42|  396964.5|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|   3|  398867.6|
|  30|  397862.0|
|  34| 401419.75|
|  59|  791453.0|
|   8| 398592.71|
|  28| 402124.62|
|  22| 402022.68|
|  85|1056336.74|
|  16| 399586.53|
+----+----------+
only showing top 20 rows

--- 0.00014019012451171875 seconds ---


In [54]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("parquetFile")

In [55]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("parquetFile")


False